## Without Augmentation

In [25]:
## Starting off with custom datasets
import os
import requests
import zipfile
from pathlib import Path

data_path =  Path('data')
image_path = data_path / 'images'

if image_path.is_dir():
    print(f"{image_path} .Direcotry already exists")
else:
    print(f"Creating direcotry. {image_path}")
    image_path.mkdir(parents=True, exist_ok=True) #* relative path
    
    #* Download the data
    with open(data_path / "Data.zip" , 'wb') as f :#* create file in write binary mode
        #* get the data from the url
        request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
        print("downloading data")
        f.write(request.content) #* write the content to the file
        
    #* Unzip the data
    with zipfile.ZipFile(data_path / "Data.zip", 'r') as zip_ref: #* read the zip file
        print('Unziping data')
        zip_ref.extractall(image_path) #* extract the data to the data_path
    

#directory = '/kaggle/working/data/'
directory = 'data/'
train_dir  = directory+'images/train'
test_dir  = directory+'images/test'
train_dir, test_dir

data\images .Direcotry already exists


('data/images/train', 'data/images/test')

In [26]:
import numpy as np
import torch 
import pathlib

from torchvision.datasets import ImageFolder, DatasetFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor, Normalize, Compose, Resize


In [27]:
train_data = ImageFolder(
    root=train_dir,
    transform=Compose([Resize((224, 224)), ToTensor()]),
    target_transform=None
)

test_data = ImageFolder(
    root=test_dir,
    transform=Compose([Resize((224, 224)), ToTensor()]),
    target_transform=None
)

In [33]:
from torch.utils.data import DataLoader
from PIL import Image


custom_train_data = DataLoader(
    dataset=train_data,
    num_workers=0,
    batch_size=1,
    shuffle=True
    
)

## Reloading Data 

In [35]:
from torchvision import datasets
simple_transfrom = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

train_data_simple  = datasets.ImageFolder(
    root=train_dir, transform=simple_transfrom, target_transform=None)

test_data_simple = datasets.ImageFolder(
    root=test_dir, transform=simple_transfrom, target_transform=None)

BATCH_SIZE =32
NUM_WORKERS = 4


dataloader_train = DataLoader(
    dataset= train_data_simple,
    batch_size=BATCH_SIZE,
    shuffle=True,
    #num_workers=NUM_WORKERS
)

dataloader_test = DataLoader(
    dataset= test_data_simple,
    batch_size=BATCH_SIZE,
    shuffle=False,
    #num_workers=NUM_WORKERS
)

In [36]:
type(os.cpu_count())

int

In [37]:
next(iter(dataloader_train))[0].shape

torch.Size([32, 3, 224, 224])

In [38]:
from torch import nn

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class TinyVGG(nn.Module):
    def __init__(self, n_channels, n_filter, n_classes)->None:
        super().__init__()
        
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=n_filter, kernel_size=3, stride=1, padding=1),
            #nn.ReLU(),
            nn.Conv2d(in_channels=n_filter, out_channels=n_filter, kernel_size=3, stride=1, padding=1),
            #nn.ReLU(),
            nn.MaxPool2d(stride=2, kernel_size=2),
            nn.LeakyReLU()
            ).to(device)
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(n_filter, n_filter, kernel_size=3, padding=1, stride=1),
            #nn.ReLU(),
            nn.Conv2d(n_filter, n_filter, stride=1, padding=1, kernel_size=3),
            #nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.LeakyReLU()
        ).to(device)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=n_filter*56*56, out_features=3),
            #nn.Softmax(dim=1)
            nn.LeakyReLU()
        ).to(device)
    
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        #print(x.shape)
        x = self.classifier(x)
        #x = x.argmax(dim=1)
        return x

In [40]:
vgg = TinyVGG(
    n_channels=3,
    n_filter=32,
    n_classes=3
)

In [41]:
import torch

In [42]:
imgs , label = next(iter(dataloader_train))
imgs
# torch.softmax(vgg(imgs), dim=1).argmax(dim=1), label
vgg(imgs.to(device)).argmax(dim=1), label

(tensor([2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
         1, 2, 2, 2, 2, 2, 2, 1], device='cuda:0'),
 tensor([2, 0, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 1, 2, 2, 1, 2, 1, 0, 0, 0, 1,
         2, 0, 1, 2, 1, 0, 2, 1]))

In [43]:
#!pip install torchinfo

In [44]:
import torchinfo
from torchinfo import summary
summary(vgg.to('cuda'), input_size=[32,3,224,224])

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 3]                   --
├─Sequential: 1-1                        [32, 32, 112, 112]        --
│    └─Conv2d: 2-1                       [32, 32, 224, 224]        896
│    └─Conv2d: 2-2                       [32, 32, 224, 224]        9,248
│    └─MaxPool2d: 2-3                    [32, 32, 112, 112]        --
│    └─LeakyReLU: 2-4                    [32, 32, 112, 112]        --
├─Sequential: 1-2                        [32, 32, 56, 56]          --
│    └─Conv2d: 2-5                       [32, 32, 112, 112]        9,248
│    └─Conv2d: 2-6                       [32, 32, 112, 112]        9,248
│    └─MaxPool2d: 2-7                    [32, 32, 56, 56]          --
│    └─LeakyReLU: 2-8                    [32, 32, 56, 56]          --
├─Sequential: 1-3                        [32, 3]                   --
│    └─Flatten: 2-9                      [32, 100352]              --
│    

In [45]:
%load_ext autoreload
%autoreload 2
from utils.training_model_fun import*

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
def accuracy_fun(y_pred, y_true):
    correct = torch.eq(y_pred,y_true).sum().item()
    return (correct / len(y_true))*100

loss_function = nn.CrossEntropyLoss()
opt = torch.optim.SGD(
    params= vgg.parameters(),
    lr = 0.01
)


In [47]:
train(
    model=vgg,
    train_dataloader=dataloader_train,
    test_dataloader=dataloader_test,
    loss_function=loss_function,
    optimizer=opt,
    epochs=1,
    device=device,
    accuracy_function=accuracy_fun,
    show_every=1,
    show_progress=True
)

Epoch: 1
------ train Acc = 33.98% || train Loss = 1.11120
------ test Acc = 54.17% || test Loss = 1.09818


{'Train_loss': [1.1112000942230225],
 'Train_acc': [33.984375],
 'Test_loss': [1.098177194595337],
 'Test_acc': [54.166666666666664]}

In [48]:
import torchvision
torch.__version__, torchvision.__version__


('1.12.1', '0.13.1')

In [1]:
!nvidia-smi

Thu Feb  9 03:49:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.98       Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   68C    P0    18W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------